In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

import pysta
import stc
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# load data

data_folder_name = "data"

# load stim and spike data
# dataset_name = "20180618"
# dataset_name = "20180621"
dataset_name = "20180626"
# dataset_name = "20180828"

stim, spike_train, info = pysta.load_data(dataset_name, data_folder_name)

# dataset_filename = "data/{}.mat".format(dataset_name)
# stim, spike_train, info = pysta.load_data_mat(dataset_filename)

channel_names = [ch.replace("ch_","") for ch in info["channel_names"]]
# info["channel_names"]

# load cell type
cell_types = pd.read_csv("{}/{}_cell_type.csv".format(data_folder_name, dataset_name))
# cell_types

['dataset_name', 'stim', 'spike_train', 'info']
(64, 9000)
(156, 9000)
{'channel_names': ['ch_12a', 'ch_12b', 'ch_12c', 'ch_13a', 'ch_13b', 'ch_13c', 'ch_13d', 'ch_13e', 'ch_14a', 'ch_14b', 'ch_14c', 'ch_14d', 'ch_16a', 'ch_16b', 'ch_16c', 'ch_17a', 'ch_17b', 'ch_17c', 'ch_17d', 'ch_21a', 'ch_21b', 'ch_21c', 'ch_21d', 'ch_21e', 'ch_22a', 'ch_22b', 'ch_23a', 'ch_23b', 'ch_23c', 'ch_23d', 'ch_25a', 'ch_25b', 'ch_25c', 'ch_26a', 'ch_26b', 'ch_26c', 'ch_26d', 'ch_27a', 'ch_27b', 'ch_27c', 'ch_27d', 'ch_28a', 'ch_28b', 'ch_28c', 'ch_31a', 'ch_31b', 'ch_31c', 'ch_31d', 'ch_32a', 'ch_32b', 'ch_32c', 'ch_33a', 'ch_33b', 'ch_33c', 'ch_33d', 'ch_35a', 'ch_35b', 'ch_35c', 'ch_35d', 'ch_35e', 'ch_35f', 'ch_35g', 'ch_36a', 'ch_36b', 'ch_36c', 'ch_36d', 'ch_37a', 'ch_37b', 'ch_37c', 'ch_37d', 'ch_37e', 'ch_38a', 'ch_41a', 'ch_41b', 'ch_41c', 'ch_41d', 'ch_42a', 'ch_42b', 'ch_42c', 'ch_43a', 'ch_43b', 'ch_43c', 'ch_45a', 'ch_45b', 'ch_45c', 'ch_46a', 'ch_46b', 'ch_47a', 'ch_48a', 'ch_48b', 'ch_48c', 

In [ ]:
# calc number of spikes
tap = 8 # -700 ms ~ 0

num_samples = list()
for idx in range(spike_train.shape[0]):    
    spike_triggered_stim, spike_count = pysta.grab_spike_triggered_stim(stim, spike_train[idx], tap)
    num_samples.append(spike_triggered_stim.shape[0])

num_samples_df = pd.DataFrame({"channel_name": channel_names, "number_of_samples": num_samples})
# num_samples_df

## check the effect of smoothing on the largest eigenvalue($\lambda_1$)

In [ ]:
# read eigenvalues

# helper function
def read_eigen_values(folder_name, channel_names):
    all_eig_values = dict()
    largest_eig_values = list()

    for channel_name in channel_names:
        filename = "{}/ch_{}_eig_val.txt".format(folder_name,channel_name)
        eig_val = np.loadtxt(filename)

        all_eig_values[channel_name] = eig_val
    #     eigen_values.append(eig_val)
        largest_eig_values.append(eig_val[0])
    return all_eig_values, largest_eig_values
    

folder_name = "{}_stc_tap{}".format(dataset_name,tap)
all_eig_values, largest_eig_values = read_eigen_values(folder_name, channel_names)
# convert to DataFrame
result_eig = pd.DataFrame({"channel_name": channel_names, "largest_eig_values":largest_eig_values})

folder_name = "{}_stc_tap{}_smoothed".format(dataset_name,tap)
all_eig_values, largest_eig_values = read_eigen_values(folder_name, channel_names)
# convert to DataFrame
result_eig_smoothed = pd.DataFrame({"channel_name": channel_names, "largest_eig_values":largest_eig_values})

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(121)
result_eig["largest_eig_values"].hist(alpha=0.75)
result_eig_smoothed["largest_eig_values"].hist(alpha=0.75)
plt.grid("off")
plt.xlabel("$\lambda_1$")
plt.ylabel("count")

plt.legend(["no smoothing", "smoothed"])

plt.subplot(122)
# merge with num_samples
num_samples_eig = num_samples_df.merge(result_eig)
num_samples_eig_smoothed = num_samples_df.merge(result_eig_smoothed)

plt.scatter(num_samples_eig["number_of_samples"], num_samples_eig["largest_eig_values"])
plt.scatter(num_samples_eig_smoothed["number_of_samples"], num_samples_eig_smoothed["largest_eig_values"])
plt.xlabel('number of samples')
plt.ylabel('$\lambda_1$')
plt.xlim(0,plt.xlim()[1])
plt.ylim(0,plt.ylim()[1])


plt.legend(["no-smoothing", "smoothed"])
# see https://arxiv.org/pdf/0901.3245.pdf to understand the effect of # samples on the largest eigenvalue

plt.savefig("eig_smoothed_{}.pdf".format(dataset_name))

## Now, let's optimize the spatial smoothing parameter $\sigma$